In [1]:
import IPython.display as ipd

import numpy as np
import torch


# import stft_64_pad_0 as stft
import stft_64 as stft
from audio_processing import griffin_lim

from scipy.io.wavfile import read
import time

In [2]:
N = 12

In [3]:
all_6_pair = {}
for i in range(N):
    for j in range(N):
        for k in range(N):
            for l in range(N):
                for t in range(N):
                    for s in range(N):
                        temp = [i,j,k,l,t,s]
                        temp.sort()
                        if tuple(temp) not in all_6_pair:
                            all_6_pair[tuple(temp)] = 0
                    
print(len(all_6_pair))
# print(all_4_pair)

all_6_arr = list(all_6_pair)
# print(all_4_arr)

12376


In [6]:
def multiple_three_dict(dic1, dic2, dic3):
    new_dic = all_6_pair.copy()
    for x1 in dic1:
        for x2 in dic2:
            for x3 in dic3:
                temp = list(x1 + x2 + x3)
                temp.sort()
                temp = tuple(temp)
                if temp not in new_dic:
                    new_dic[temp] = 0
                new_dic[temp] += (dic1[x1]*dic2[x2])
    return new_dic

# multiple_three_dict({(0,1):1}, {(0,2):1}, {(3,2):1})

In [8]:
dic_arr = []
for k in range(N//2):
    dic_arr.append({})
    for i in range(N):
        i_plus_k = (i+k)%N
        temp =[i, i_plus_k]
        temp.sort()
        temp = tuple(temp)
        dic_arr[-1][temp] = 1
        
dic_arr.append({})
for i in range(N//2):
    i_plus_k = (i+N//2)%N
    temp =[i, i_plus_k]
    temp.sort()
    temp = tuple(temp)
    dic_arr[-1][temp] = 1

# for k in range(N//2+1):
#     print(dic_arr[k])

In [11]:
dic_arr_6 = []
for i in range(N//2+1):
    for j in range(i+1):
        for k in range(j+1):
            dic_arr_6.append(multiple_three_dict(dic_arr[i], dic_arr[j], dic_arr[k]))
print(len(dic_arr_6))
# print(len(dic_arr_6[0]))
# print(dic_arr_6[0])

84


In [12]:

target = all_6_pair.copy()
target_arr = [[0,3,6,9],
              [1,4,7,10],
              [2,5,8,11],
             ]

for i in target_arr[0]:
    for j in target_arr[0]:
        for k in target_arr[1]:
            for l in target_arr[1]:
                for s in target_arr[2]:
                    for t in target_arr[2]:
                        temp = [i,j,k,l,s,t]
                        temp.sort()
                        target[tuple(temp)] += 1


In [14]:
A = np.zeros((len(all_6_pair), len(dic_arr_6)))
b = np.zeros((len(all_6_pair),1))

for i in range(len(all_6_arr)):
    b[i,0] = target[all_6_arr[i]]
# for i in range(len(all_4_arr)):
#     b[i,0] = dic_arr_4[0][all_4_arr[i]]+ 2*dic_arr_4[2][all_4_arr[i]]
print(np.sum(b>0))
print(A.shape)
print(b.shape)

for k in range(len(dic_arr_6)):
    for i in range(len(all_6_arr)):
        A[i,k] = dic_arr_6[k][all_6_arr[i]]

1000
(12376, 84)
(12376, 1)


In [15]:
Q, R = np.linalg.qr(A, mode='reduced')
Qb = np.matmul(Q.T, b)
ans = np.linalg.solve(R,Qb)

b_hat = np.matmul(A, ans)
print(np.sum(np.abs(b_hat-b)))
print(A.shape)
print(ans.T)

7932.977563261526
(12376, 84)
[[ 0.00819958 -0.0189092  -0.02763942  0.07481893 -0.01965292 -0.06361
   0.24487903 -0.02469156  0.2480829   0.07498357  0.0717545  -0.12221869
  -0.03550547 -0.12277246 -0.10398843 -0.0425419   0.18342643 -0.16360713
  -0.15770383  0.14951696 -0.01975522 -0.06372119  0.2487538  -0.06057565
   0.52044325  0.23701192 -0.12263374 -0.10491656 -0.10637152 -0.15775512
  -0.0247401   0.24797318  0.2414679  -0.04162788  0.07402268 -0.0189092
  -0.0657821   0.24022694 -0.06361     0.52263024  0.2480829  -0.12221869
  -0.09425934 -0.10398843 -0.16360713 -0.06372119  0.51373389  0.52044325
  -0.10491656  0.24797318 -0.02763942  0.24022694  0.24487903 -0.03550547
   0.2487538   0.07481893  0.0784389  -0.13251896 -0.04260971 -0.13582908
  -0.10655377 -0.03811832  0.41663441 -0.42281901 -0.42232648  0.53034873
  -0.13599683 -0.10848659 -0.0847974  -0.42246927 -0.03609934 -0.13251896
  -0.09418236 -0.10655377 -0.42281901 -0.10848659 -0.04260971  0.16957702
  -0.1005996